<a href="https://colab.research.google.com/github/MEET4150/TIMELINE-WORKING/blob/main/MICROSOFT_TrOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pillow pandas matplotlib seaborn python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import os
import time
import difflib
import pandas as pd
from Levenshtein import distance as levenshtein_distance
import matplotlib.pyplot as plt
import seaborn as sns
import argparse
from google.colab import files
import io

In [ ]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")

# Print the model architecture
print(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.48.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-23): 24 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=False)
              (key): Linear(in_features=1024, out_features=1024, bias=False)
              (value): Linear(in_features=1024, out_features=1024, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dens

In [ ]:
def setup_ocr_model(model_name='microsoft/trocr-large-handwritten'):
    """
    Initialize the TrOCR model and processor from Hugging Face.

    Args:
        model_name (str): Name of the model to use from Hugging Face

    Returns:
        tuple: (processor, model) - the initialized OCR processor and model
    """
    # Load the processor and model
    processor = TrOCRProcessor.from_pretrained(model_name)
    model = VisionEncoderDecoderModel.from_pretrained(model_name)

    # Set device to GPU if available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    print(f"Model '{model_name}' loaded and running on {device}")
    return processor, model

def perform_ocr(image_path, processor, model):
    """
    Perform OCR on the given image.

    Args:
        image_path (str): Path to the input image
        processor: The TrOCR processor
        model: The TrOCR model

    Returns:
        str: The extracted text from the image
    """
    try:
        # Load the image
        image = Image.open(image_path).convert("RGB")

        # Process image and generate prediction
        pixel_values = processor(images=image, return_tensors="pt").pixel_values

        # Move input to the same device as model
        device = next(model.parameters()).device
        pixel_values = pixel_values.to(device)

        # Generate text
        generated_ids = model.generate(pixel_values)

        # Decode the generated IDs to text
        predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return predicted_text

    except Exception as e:
        return f"Error processing image: {str(e)}"

In [ ]:
model.summary()



NameError: name 'model' is not defined

In [ ]:
class OCRModelEvaluator:
    def __init__(self, model_name='microsoft/trocr-base-handwritten'):
        """
        Initialize the OCR model evaluator with a specific model.

        Args:
            model_name (str): Name of the Hugging Face model to use
        """
        # Load the processor and model
        self.processor = TrOCRProcessor.from_pretrained(model_name)
        self.model = VisionEncoderDecoderModel.from_pretrained(model_name)

        # Set device to GPU if available, otherwise use CPU
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        print(f"Model '{model_name}' loaded and running on {self.device}")

        # Initialize metrics storage
        self.results = []

    def perform_ocr(self, image_path):
        """Perform OCR on a single image and return the predicted text."""
        try:
            # Load and preprocess the image
            image = Image.open(image_path).convert("RGB")

            # Process image
            pixel_values = self.processor(images=image, return_tensors="pt").pixel_values
            pixel_values = pixel_values.to(self.device)

            # Start timing
            start_time = time.time()

            # Generate text
            generated_ids = self.model.generate(pixel_values)

            # End timing
            process_time = time.time() - start_time

            # Decode the generated IDs to text
            predicted_text = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

            return predicted_text, process_time

        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")
            return None, 0

    def calculate_metrics(self, predicted_text, ground_truth):
        """Calculate various text similarity metrics."""
        if predicted_text is None:
            return {
                'character_error_rate': 1.0,
                'word_accuracy': 0.0,
                'levenshtein_distance': len(ground_truth) if ground_truth else 0,
                'similarity_ratio': 0.0
            }

        # Character Error Rate (CER)
        if len(ground_truth) > 0:
            cer = levenshtein_distance(predicted_text, ground_truth) / len(ground_truth)
        else:
            cer = 1.0 if predicted_text else 0.0

        # Word Accuracy
        pred_words = predicted_text.split()
        truth_words = ground_truth.split()

        if len(truth_words) > 0:
            correct_words = sum(1 for pw, tw in zip(pred_words, truth_words) if pw == tw)
            word_acc = correct_words / len(truth_words)
        else:
            word_acc = 0.0

        # Similarity ratio using difflib
        similarity = difflib.SequenceMatcher(None, predicted_text, ground_truth).ratio()

        return {
            'character_error_rate': cer,
            'word_accuracy': word_acc,
            'levenshtein_distance': levenshtein_distance(predicted_text, ground_truth),
            'similarity_ratio': similarity
        }

    def evaluate_dataset(self, dataset_dir, ground_truth_file):
        """
        Evaluate the OCR model on a dataset of images with known ground truth.

        Args:
            dataset_dir (str): Directory containing the images
            ground_truth_file (str): Path to CSV file with columns: 'image_file', 'ground_truth'

        Returns:
            DataFrame with evaluation results
        """
        # Load ground truth data
        try:
            ground_truth_df = pd.read_csv(ground_truth_file)
        except Exception as e:
            print(f"Error loading ground truth file: {str(e)}")
            return None

        # Process each image
        for index, row in ground_truth_df.iterrows():
            image_path = os.path.join(dataset_dir, row['image_file'])

            if not os.path.exists(image_path):
                print(f"Warning: Image {image_path} does not exist. Skipping.")
                continue

            # Perform OCR
            predicted_text, process_time = self.perform_ocr(image_path)

            # Calculate metrics
            metrics = self.calculate_metrics(predicted_text, row['ground_truth'])

            # Store results
            result = {
                'image_file': row['image_file'],
                'ground_truth': row['ground_truth'],
                'predicted_text': predicted_text,
                'processing_time': process_time,
                **metrics
            }
            self.results.append(result)

            # Print progress
            if (index + 1) % 10 == 0:
                print(f"Processed {index + 1}/{len(ground_truth_df)} images")

        # Convert results to DataFrame
        results_df = pd.DataFrame(self.results)
        return results_df

    def get_overall_metrics(self):
        """Calculate overall performance metrics across all evaluated images."""
        if not self.results:
            return None

        results_df = pd.DataFrame(self.results)

        overall_metrics = {
            'total_images': len(results_df),
            'avg_character_error_rate': results_df['character_error_rate'].mean(),
            'avg_word_accuracy': results_df['word_accuracy'].mean(),
            'avg_similarity_ratio': results_df['similarity_ratio'].mean(),
            'avg_processing_time': results_df['processing_time'].mean(),
            'failed_images': results_df['predicted_text'].isna().sum()
        }

        return overall_metrics

    def visualize_results(self, save_path=None):
        """Generate visualizations of the evaluation results."""
        if not self.results:
            print("No results to visualize")
            return

        results_df = pd.DataFrame(self.results)

        # Set up the figure
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # 1. Distribution of Character Error Rate
        sns.histplot(results_df['character_error_rate'], kde=True, ax=axes[0, 0])
        axes[0, 0].set_title('Distribution of Character Error Rate')
        axes[0, 0].set_xlabel('Character Error Rate')
        axes[0, 0].set_ylabel('Count')

        # 2. Distribution of Word Accuracy
        sns.histplot(results_df['word_accuracy'], kde=True, ax=axes[0, 1])
        axes[0, 1].set_title('Distribution of Word Accuracy')
        axes[0, 1].set_xlabel('Word Accuracy')
        axes[0, 1].set_ylabel('Count')

        # 3. Processing Time vs Image File (to see if certain images take longer)
        sns.scatterplot(data=results_df, x=results_df.index, y='processing_time', ax=axes[1, 0])
        axes[1, 0].set_title('Processing Time per Image')
        axes[1, 0].set_xlabel('Image Index')
        axes[1, 0].set_ylabel('Processing Time (seconds)')

        # 4. Similarity Ratio Distribution
        sns.histplot(results_df['similarity_ratio'], kde=True, ax=axes[1, 1])
        axes[1, 1].set_title('Distribution of Similarity Ratio')
        axes[1, 1].set_xlabel('Similarity Ratio')
        axes[1, 1].set_ylabel('Count')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path)
            print(f"Visualization saved to {save_path}")

        plt.show()

# PART 4: Google Colab Helper Functions
# -------------------------------------
def upload_images_to_colab():
    """
    Helper function to upload images to Colab and create a directory

    Returns:
        str: Path to the directory containing the uploaded images
    """
    # Create a directory for the uploaded images
    os.makedirs('uploaded_images', exist_ok=True)

    print("Please upload your image files...")
    uploaded = files.upload()

    for filename, content in uploaded.items():
        with open(os.path.join('uploaded_images', filename), 'wb') as f:
            f.write(content)

    print(f"Uploaded {len(uploaded)} images to 'uploaded_images' directory")
    return 'uploaded_images'

def create_sample_ground_truth(image_dir):
    """
    Create a sample ground truth CSV file for the uploaded images

    Args:
        image_dir (str): Directory containing the images

    Returns:
        str: Path to the created ground truth CSV file
    """
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    data = {
        'image_file': image_files,
        'ground_truth': [''] * len(image_files)  # Empty ground truth to be filled by user
    }

    df = pd.DataFrame(data)
    csv_path = 'ground_truth_template.csv'
    df.to_csv(csv_path, index=False)

    print(f"Created template CSV at {csv_path}")
    print("Please download this file, fill in the ground truth text for each image, and upload it back")

    # Download the template
    files.download(csv_path)
    return csv_path

def upload_ground_truth():
    """
    Helper function to upload the filled ground truth CSV

    Returns:
        str: Path to the uploaded ground truth CSV
    """
    print("Please upload your filled ground truth CSV file...")
    uploaded = files.upload()

    if len(uploaded) != 1:
        print("Error: Please upload exactly one CSV file")
        return None

    filename = list(uploaded.keys())[0]
    if not filename.endswith('.csv'):
        print(f"Warning: {filename} is not a CSV file")

    csv_path = f'uploaded_{filename}'
    with open(csv_path, 'wb') as f:
        f.write(uploaded[filename])

    print(f"Ground truth CSV saved to {csv_path}")
    return csv_path

# PART 5: Main Functions for Single Image OCR and Evaluation
# ---------------------------------------------------------
def run_single_image_ocr():
    """Interactive function to run OCR on a single uploaded image"""
    # Setup model
    processor, model = setup_ocr_model()

    # Upload image
    print("Please upload an image to perform OCR:")
    uploaded = files.upload()

    if not uploaded:
        print("No image uploaded")
        return

    # Get the first uploaded image
    image_name = next(iter(uploaded))
    image_path = f"/content/{image_name}"

    with open(image_path, 'wb') as f:
        f.write(uploaded[image_name])

    # Perform OCR
    text = perform_ocr(image_path, processor, model)

    # Display the image
    image = Image.open(image_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.title("Uploaded Image")
    plt.show()

    # Display the extracted text
    print("\nExtracted Text:")
    print("--------------")
    print(text)

    # Clean up
    os.remove(image_path)

def run_evaluation_workflow():
    """Run the complete evaluation workflow in Colab"""
    # 1. Upload images
    image_dir = upload_images_to_colab()

    # 2. Create and download ground truth template
    create_sample_ground_truth(image_dir)

    # 3. Wait for user to upload filled ground truth
    ground_truth_path = upload_ground_truth()

    if ground_truth_path is None:
        print("Evaluation cancelled: No ground truth CSV uploaded")
        return

    # 4. Initialize evaluator
    print("\nInitializing OCR model for evaluation...")
    evaluator = OCRModelEvaluator()

    # 5. Run evaluation
    print("\nRunning evaluation. This may take some time depending on the number of images...")
    results = evaluator.evaluate_dataset(image_dir, ground_truth_path)

    if results is None:
        print("Evaluation failed")
        return

    # 6. Show overall metrics
    overall = evaluator.get_overall_metrics()
    print("\nOverall Performance Metrics:")
    for metric, value in overall.items():
        print(f"{metric}: {value}")

    # 7. Generate and show visualizations
    print("\nGenerating visualizations...")
    evaluator.visualize_results('performance_plot.png')

    # 8. Save and offer to download results
    results_path = 'ocr_evaluation_results.csv'
    results.to_csv(results_path, index=False)
    print(f"\nResults saved to {results_path}")
    print("You can download the results using the following code:")
    print("from google.colab import files")
    print("files.download('ocr_evaluation_results.csv')")
    print("files.download('performance_plot.png')")

# PART 6: Colab Interactive Menu
# ------------------------------
def display_interactive_menu():
    """
    Display an interactive menu for the user to choose what to do
    """
    print("\n==== OCR Model and Evaluation Tool ====")
    print("1. Run OCR on a single image")
    print("2. Run complete evaluation workflow")
    print("3. Exit")

    choice = input("\nEnter your choice (1-3): ")

    if choice == '1':
        run_single_image_ocr()
    elif choice == '2':
        run_evaluation_workflow()
    elif choice == '3':
        print("Exiting...")
        return
    else:
        print("Invalid choice. Please try again.")

    # Show menu again
    display_interactive_menu()

# Main Execution
if __name__ == "__main__":
    # Show interactive menu when run in Colab
    display_interactive_menu()


==== OCR Model and Evaluation Tool ====
1. Run OCR on a single image
2. Run complete evaluation workflow
3. Exit


In [ ]:
from google.colab import files
files.download('ocr_evaluation_results.csv')  # Downloads the results table
files.download('performance_plot.png')

# New Section